<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/mthree-c422-dipti/Exercises/day-7/Feature_Engineerring_with_Validation/FE_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Feature Engineering and Validation Pipeline with the Titanic Dataset
Build an automated, scalable pipeline in Google Colab that performs feature engineering, data validation, and outputs a clean dataset ready for modeling.

**Objectives**
- Load and split the Titanic dataset.

- Apply feature engineering (including new features and encoding).

- Implement validation checks to ensure data quality.

- Modularize the steps into reusable functions.

- Demonstrate end-to-end pipeline execution.

In [1]:
# Step 1: Install and Import Libraries
!pip install pandas numpy scikit-learn -q

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.exceptions import DataConversionWarning
import warnings

# Suppress warnings for clearer output
warnings.filterwarnings(action='ignore', category=DataConversionWarning)


In [2]:
# Step 2: Load and Split the Dataset
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Split into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Survived'])


In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
692,693,1,3,"Lam, Mr. Ali",male,NaN,0,0,1601,56.4958,NaN,S
481,482,0,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0000,NaN,S
527,528,0,1,"Farthing, Mr. John",male,NaN,0,0,PC 17483,221.7792,C95,S
855,856,1,3,"Aks, Mrs. Sam (Leah Rosen)",female,18.0,0,1,392091,9.3500,NaN,S
801,802,1,2,"Collyer, Mrs. Harvey (Charlotte Annie Tate)",female,31.0,1,1,C.A. 31921,26.2500,NaN,S


In [4]:
val_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
565,566,0,3,"Davies, Mr. Alfred J",male,24.0,2,0,A/4 48871,24.1500,NaN,S
160,161,0,3,"Cribb, Mr. John Hatfield",male,44.0,0,1,371362,16.1000,NaN,S
553,554,1,3,"Leeni, Mr. Fahim (""Philip Zenni"")",male,22.0,0,0,2620,7.2250,NaN,C
860,861,0,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,NaN,S
241,242,1,3,"Murphy, Miss. Katherine ""Kate""",female,NaN,1,0,367230,15.5000,NaN,Q


In [5]:
# Step 3: Define the Pipeline Functions
def clean_data(df):
    df = df.drop(columns=["PassengerId","Ticket","Cabin"], errors="ignore").copy()
    # Impute Age and Embarked
    df["Age"] = df["Age"].fillna(df["Age"].median())
    df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])
    return df

def engineer_features(df):
    df = df.copy()
    # Title extraction
    df["Title"] = df["Name"].str.extract(r",\s*([^\.]+)\.")
    rare_titles = ["Lady","Countess","Capt","Col","Don","Dr","Major","Rev","Sir","Jonkheer","Dona"]
    df["Title"] = df["Title"].replace(rare_titles, "Rare")
    # Family size & is alone
    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)
    # Fare and Age bins
    df["FareBin"] = pd.qcut(df["Fare"].fillna(0), 4, labels=False)
    df["AgeBin"]  = pd.cut(df["Age"], bins=[0,12,20,40,60,100], labels=False)
    # Drop unused columns
    df = df.drop(columns=["Name","SibSp","Parch"])
    # One-hot encode
    df = pd.get_dummies(df, columns=["Sex","Embarked","Title"], drop_first=True)
    return df

def validate_data(df):
    errors = []
    # Check for nulls
    null_counts = df.isnull().sum()
    if null_counts.any():
        errors.append(f"Null values found:\n{null_counts[null_counts>0]}")
    # Check expected columns
    expected_cols = {"Survived","Pclass","Age","Fare","FamilySize","IsAlone","FareBin","AgeBin"}
    missing = expected_cols - set(df.columns)
    if missing:
        errors.append(f"Missing columns: {missing}")
    return errors


In [6]:
# Step 4: Execute the Pipeline
# Clean and feature-engineer training data
train_clean = clean_data(train_df)
train_feat  = engineer_features(train_clean)
train_errors = validate_data(train_feat)
print("Train validation errors:", train_errors or "None")

# Clean and feature-engineer validation data
val_clean = clean_data(val_df)
val_feat  = engineer_features(val_clean)
val_errors = validate_data(val_feat)
print("Validation validation errors:", val_errors or "None")


Train validation errors: None
Validation validation errors: None


In [7]:
# Step 5: Save Prepared Data
train_feat.to_csv("titanic_train_prepared.csv", index=False)
val_feat.to_csv("titanic_val_prepared.csv", index=False)


In [9]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Reflection Questions
- How did validation checks help catch data quality issues early?
Validation checks are crucial for detecting problems in raw data before they can propagate downstream and cause model errors or misleading insights. In the code, examples of validation include:

fillna(0) for Fare: Handles missing values gracefully instead of letting them crash a model or cause NaNs in engineered features.

Binning age (pd.cut) and fare (pd.qcut): Would fail if Age or Fare had invalid or non-numeric values, which triggers you to check and clean.

Title extraction from Name: If the regex returns unexpected results (e.g., NaN), it signals inconsistencies in the name format.

Use of .astype(int) for IsAlone: Forces boolean logic into consistent numeric types.

- Which engineered features contributed most to dataset richness?
Feature Name	Value Added
Title	Encodes social status, which correlates with survival or credit behavior.
FamilySize	Combines SibSp + Parch into a meaningful feature of group dynamics.
IsAlone	Binary simplification of family size, often predictive in behavioral models.
FareBin, AgeBin	Turn continuous values into distribution-aware categories, useful for tree models or risk segmenting.
One-hot encoding	Translates categorical features like Sex, Embarked, Title into model-friendly binary vectors.

- How would you extend this pipeline to include scaling, imputation strategies, or integrate with a model training step?



In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_columns = ['Fare', 'Age', 'FamilySize', 'avg_bill_amt', 'avg_pay_amt', 'pay_bill_ratio']  # example

df[scaled_columns] = scaler.fit_transform(df[scaled_columns])


In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  # or 'median'/'most_frequent'
df[['Age']] = imputer.fit_transform(df[['Age']])


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier())
])

pipeline.fit(X_train, y_train)
